In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# more necessary imports
from tqdm import tqdm, trange
import time
from pathlib import Path
import dask.dataframe as dd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler


#import lightgbm as lgb
from optuna.integration import lightgbm as lgb

import glob

In [ ]:
# number of train and test files
trains = []
tests = []
for file in os.listdir("/kaggle/input/predict-volcanic-eruptions-ingv-oe/train/"):
    trains.append(file)
    
for file in os.listdir("/kaggle/input/predict-volcanic-eruptions-ingv-oe/test/"):
    tests.append(file)
    
print(f"Number of train files: {len(trains)}")
print(f"Number of test files: {len(tests)}")

# **Feature Extraction**
Code from [Volcano](https://www.kaggle.com/ymdhryk/volcano/notebook#Feature-Extraction) notebook by [ymdhryk](https://www.kaggle.com/ymdhryk)

In [ ]:
# Define feature extraction function

fs = ["_mean","_std","_max","_min","_mad","_skew","_kurt","_nunique",
      "_quantile_05","_quantile_10","_quantile_30","_quantile_70","_quantile_90","_quantile_95",
      "_fft_power_mean","_fft_power_std","_fft_power_min","_fft_power_max",
      "_fft_power_sum_low","_fft_power_sum_middle","_fft_power_sum_high",
      "_fft_power_mad","_fft_power_skew","_fft_power_kurt","_fft_power_nunique",
      "_fft_power_quantile_05","_fft_power_quantile_10","_fft_power_quantile_30","_fft_power_quantile_70","_fft_power_quantile_90","_fft_power_quantile_95",
      "_cross_0_count",
      "_roll_mean_min","_roll_mean_max","_roll_dist_min","_roll_dist_max","_roll_dist_diff_min","_roll_dist_diff_max",
      #"_first_005","_last_005","_first_010","_last_010","_first_030","_last_030","_first_070","_last_070","_first_090","_last_090","_first_095","_last_095",
      #"_abs_0250_min","_abs_0250_max","_abs_0500_min","_abs_0500_max","_abs_0750_min","_abs_0750_max","_abs_1000_min","_abs_1000_max","_abs_1250_min","_abs_1250_max","_abs_1500_min","_abs_1500_max",
     ]

def extract(segment_id,dir_="train"):
    """
    Extract statistical features for each sensor signal
    
    - Mean
    - Standard Deviation
    - Maximum
    - Minimum
    - Mean Absolute Deviation
    - Skewness
    - Kurtosis
    - Median
    - Mode
    - (Unbiased) Standard Error of the Mean
    - Number of Unique Values
    """
    #display(segment_id)
    f = pd.read_csv(f"/kaggle/input/predict-volcanic-eruptions-ingv-oe/{dir_}/{segment_id}.csv")
    
    # Fill NaN
    f.interpolate(axis=0,inplace=True)
    #display(f)
    
    # Quantile
    q005 = f.quantile(0.05,axis=0)
    q010 = f.quantile(0.1 ,axis=0)
    q030 = f.quantile(0.3 ,axis=0)
    q070 = f.quantile(0.7 ,axis=0)
    q090 = f.quantile(0.9 ,axis=0)
    q095 = f.quantile(0.95,axis=0)
    
    # Rolling
    roll = f.rolling(500)
    roll_mean = roll.mean()
    roll_max = roll.max()
    roll_min = roll.min()
    roll_dist = roll_max - roll_min
    roll_dist_diff = roll_dist.diff()

    # FFT power
    # Remove under flowed 0 frequency and mirrored higher half.
    fft = pd.DataFrame(np.fft.fft(f.fillna(0)),columns=f.columns).abs().iloc[1:30001,:]
    fft[f.iloc[1:30001,:].isnull()] = np.nan
    #display(fft)
    
    # Timing information inside 10 minute.
    f005 = f.where(f < q005)
    f010 = f.where(f < q010)
    f030 = f.where(f < q030)

    f070 = f.where(f > q070)
    f090 = f.where(f > q090)
    f095 = f.where(f > q095)
    
    f_abs = f.abs()
    f_abs_0250 = f_abs.where(f_abs >  250)
    f_abs_0500 = f_abs.where(f_abs >  500)
    f_abs_0750 = f_abs.where(f_abs >  750)
    f_abs_1000 = f_abs.where(f_abs > 1000)
    f_abs_1250 = f_abs.where(f_abs > 1250)
    f_abs_1500 = f_abs.where(f_abs > 1500)


    return pd.concat((f.mean(axis=0).add_suffix("_mean"),
                      f.std(axis=0).add_suffix("_std"),
                      f.max(axis=0).add_suffix("_max"),
                      f.min(axis=0).add_suffix("_min"),
                      f.mad(axis=0).add_suffix("_mad"),
                      f.skew(axis=0).add_suffix("_skew"),
                      f.kurt(axis=0).add_suffix("_kurt"),
                      f.nunique(axis=0).add_suffix("_nunique"),
                      q005.add_suffix("_quantile_05"),
                      q010.add_suffix("_quantile_10"),
                      q030.add_suffix("_quantile_30"),
                      q070.add_suffix("_quantile_70"),
                      q090.add_suffix("_quantile_90"),
                      q095.add_suffix("_quantile_95"),
                      fft.mean(axis=0).add_suffix("_fft_power_mean"),
                      fft.std(axis=0).add_suffix("_fft_power_std"),
                      fft.min(axis=0).add_suffix("_fft_power_min"),
                      fft.max(axis=0).add_suffix("_fft_power_max"),
                      fft.iloc[:10000,:].sum(axis=0).add_suffix("_fft_power_sum_low"),
                      fft.iloc[10000:20000,:].sum(axis=0).add_suffix("_fft_power_sum_middle"),
                      fft.iloc[20000:,:].sum(axis=0).add_suffix("_fft_power_sum_high"),
                      fft.mad(axis=0).add_suffix("_fft_power_mad"),
                      fft.skew(axis=0).add_suffix("_fft_power_skew"),
                      fft.kurt(axis=0).add_suffix("_fft_power_kurt"),
                      fft.nunique(axis=0).add_suffix("_fft_power_nunique"),
                      fft.quantile(0.05,axis=0).add_suffix("_fft_power_quantile_05"),
                      fft.quantile(0.1,axis=0).add_suffix("_fft_power_quantile_10"),
                      fft.quantile(0.3,axis=0).add_suffix("_fft_power_quantile_30"),
                      fft.quantile(0.7,axis=0).add_suffix("_fft_power_quantile_70"),
                      fft.quantile(0.9,axis=0).add_suffix("_fft_power_quantile_90"),
                      fft.quantile(0.95,axis=0).add_suffix("_fft_power_quantile_95"),
                      ((f * f.shift()) < 0).sum(axis=0).add_suffix("_cross_0_count"),
                      roll_mean.min(axis=0).add_suffix("_roll_mean_min"),
                      roll_mean.max(axis=0).add_suffix("_roll_mean_max"),
                      roll_dist.min(axis=0).add_suffix("_roll_dist_min"),
                      roll_dist.max(axis=0).add_suffix("_roll_dist_max"),
                      roll_dist_diff.min(axis=0).add_suffix("_roll_dist_diff_min"),
                      roll_dist_diff.max(axis=0).add_suffix("_roll_dist_diff_max"),
                      f005.idxmin().add_suffix("_first_005"),
                      f005.idxmax().add_suffix("_last_005"),
                      f010.idxmin().add_suffix("_first_010"),
                      f010.idxmax().add_suffix("_last_010"),
                      f030.idxmin().add_suffix("_first_030"),
                      f030.idxmax().add_suffix("_last_030"),
                      f070.idxmin().add_suffix("_first_070"),
                      f070.idxmax().add_suffix("_last_070"),
                      f090.idxmin().add_suffix("_first_090"),
                      f090.idxmax().add_suffix("_last_090"),
                      f095.idxmin().add_suffix("_first_095"),
                      f095.idxmax().add_suffix("_last_095"),
                      f_abs_0250.idxmin().add_suffix("_abs_0250_min"),
                      f_abs_0250.idxmax().add_suffix("_abs_0250_max"),
                      f_abs_0500.idxmin().add_suffix("_abs_0500_min"),
                      f_abs_0500.idxmax().add_suffix("_abs_0500_max"),
                      f_abs_0750.idxmin().add_suffix("_abs_0750_min"),
                      f_abs_0750.idxmax().add_suffix("_abs_0750_max"),
                      f_abs_1000.idxmin().add_suffix("_abs_1000_min"),
                      f_abs_1000.idxmax().add_suffix("_abs_1000_max"),
                      f_abs_1250.idxmin().add_suffix("_abs_1250_min"),
                      f_abs_1250.idxmax().add_suffix("_abs_1250_max"),
                      f_abs_1500.idxmin().add_suffix("_abs_1500_min"),
                      f_abs_1500.idxmax().add_suffix("_abs_1500_max"),
                     ),
                     axis=0)

In [ ]:
# Test with small data

%time small_features = sorted_train.iloc[[0,1,2],:]["segment_id"].apply(extract)

display(small_features)

frame = small_features.iloc[:0]

In [ ]:
#Extract features for train data

train = pd.read_csv("/kaggle/input/predict-volcanic-eruptions-ingv-oe/train.csv")
sorted_train = train.sort_values("time_to_eruption",ascending=False)

%time features = dd.from_pandas(sorted_train["segment_id"],npartitions=4).apply(extract,meta=frame).compute(scheduler="processes")

data = pd.concat((sorted_train,features),axis=1)

# Save features to reuse
data.to_csv("train_data.csv")

data

In [ ]:
# Extract features for test data

test_ = pd.DataFrame([os.path.basename(f)[:-4] for f in glob.glob('/kaggle/input/predict-volcanic-eruptions-ingv-oe/test/*')], columns=["segment_id"])

%time _test = dd.from_pandas(test_["segment_id"],npartitions=4).apply(extract,dir_="test",meta=frame).compute(scheduler="processes")
test = pd.concat((test_,_test),axis=1)

# Save features to reuse
test.to_csv("test_data.csv")

test

# **LightGBM Regressor**
Code from [Introduction to Volcanology, Seismograms and LGBM](https://www.kaggle.com/jesperdramsch/introduction-to-volcanology-seismograms-and-lgbm#Train-a-LightGBM-Regressor) notebook by [jesperdramsch](https://www.kaggle.com/jesperdramsch)

In [ ]:
import lightgbm as lgbm
from sklearn.model_selection import KFold
import gc

n_fold = 7
folds = KFold(n_splits=n_fold, shuffle=True, random_state=101)
features = list(data.drop(["segment_id", "time_to_eruption"], axis=1).columns)
target_name = ["time_to_eruption"]

params = {
    "n_estimators": 2000,
    "boosting_type": "gbdt",
    "metric": "mae",
    "num_leaves": 66,
    "learning_rate": 0.005,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "agging_freq": 3,
    "max_bins": 2048,
    "verbose": 0,
    "random_state": 101,
    "nthread": -1,
    "device": "gpu",
}

sub_preds = np.zeros(test.shape[0])
feature_importance = pd.DataFrame(index=list(range(n_fold)), columns=features)

for n_fold, (trn_idx, val_idx) in enumerate(folds.split(data)):
    print(f"Fold {n_fold}:")
    trn_x, trn_y = data[features].iloc[trn_idx], data[target_name].iloc[trn_idx]
    val_x, val_y = data[features].iloc[val_idx], data[target_name].iloc[val_idx]
    
    model = lgbm.LGBMRegressor(**params)
    
    model.fit(trn_x, trn_y, 
            eval_set= [(trn_x, trn_y), (val_x, val_y)], 
            eval_metric="mae", verbose=200, early_stopping_rounds=50
           )

    feature_importance.iloc[n_fold, :] = model.feature_importances_
    
    sub_preds += model.predict(test[features], num_iteration=model.best_iteration_) / folds.n_splits

# Submission file

In [ ]:
submission = pd.DataFrame()
submission['segment_id'] = test["segment_id"]
submission['time_to_eruption'] = sub_preds
submission.to_csv('submission.csv', header=True, index=False)

submission